In [1]:
!pip install -q git+https://github.com/tcstrength/item-identification.git@main timm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import pandas as pd
from pathlib import Path
from hcmus.core import appconfig
from hcmus.lbs import LabelStudioConnector

2025-06-08 21:56:59.517 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
lsb_connector = LabelStudioConnector(
    url=appconfig.LABEL_STUDIO_URL,
    api_key=appconfig.LABEL_STUDIO_API_KEY,
    project_id=appconfig.LABEL_STUDIO_PROJECT_MAPPING["train"],
    temp_dir=appconfig.LABEL_STUDIO_TEMP_DIR
)

In [5]:
tasks = lsb_connector.get_tasks()
labels = lsb_connector.extract_labels(tasks)
dataset = lsb_connector.download_dataset(tasks, labels)
dataset = [x for x in dataset if x.get("target").get("labels")]

2025-06-08 21:57:00.624 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:121 - Page size is too large, only 3375 tasks available.
2025-06-08 21:57:00.625 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:125 - New `page_to` applied: 34
Loading tasks: 100%|██████████| 34/34 [00:08<00:00,  3.83it/s]


In [6]:
def build_classification_dataset(dataset, labels):
    ret_dataset = []
    unknown_idx = labels["unknown"]
    prioritized_label = "8935136865648"
    prioritized_idx = [v for k, v in labels.items() if k.startswith(prioritized_label)][0]

    for item in dataset:
        target_labels = list(set(item.get("target").get("labels")))
        if unknown_idx in labels:
            labels.remove(unknown_idx)

        final_label = -1
        if len(target_labels) == 1:
            final_label = target_labels[0]

        if len(target_labels) > 1:
            for label in item.get("target").get("labels"):
                final_label = label
                if prioritized_idx in target_labels:
                    final_label = prioritized_idx
                    break

        ret_dataset.append({
            "image": item.get("image"),
            "label": final_label
        })
    return ret_dataset

def select_labels_m_samples(cls_dataset, m_samples=12):
    label_stats = [x.get("label") for x in cls_dataset]
    df = pd.DataFrame(label_stats, columns=["label"])
    df = df.groupby("label")["label"].agg(["count"])
    df = df.sort_values("count")
    df = df[df["count"] >= m_samples]
    df = df.reset_index()
    selected_label_idx = df["label"].tolist()
    return selected_label_idx

def filter_cls_dataset(cls_dataset, selected_labels):
    ret_dataset = []
    for item in cls_dataset:
        if item.get("label") in selected_labels:
            ret_dataset.append(item)
    return ret_dataset

In [7]:
cls_dataset = build_classification_dataset(dataset, labels)
selected_labels = select_labels_m_samples(cls_dataset, 12)
filtered_dataset = filter_cls_dataset(cls_dataset, selected_labels)

In [8]:
idx2label = {v: k for k, v in labels.items()}

In [9]:
dataset_dir = Path("dataset")
dataset_dir.mkdir(exist_ok=True)

for item in filtered_dataset:
    label = item.get("label")
    path = item.get("image")
    label_name = idx2label[label]
    label_name = label_name.replace("/", "-")
    label_name = label_name.replace("'", "-")
    label_dir = dataset_dir.joinpath(label_name)
    label_dir.mkdir(exist_ok=True)
    return_code = os.system(f"cp -f '{path}' '{label_dir}'")
    if return_code != 0:
        print(label_name)
        break

### Models

In [10]:
# Install timm for EfficientNet
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from timm import create_model
from tqdm import tqdm

# Configuration
DATA_DIR = 'dataset'  # 👈 Path to your single dataset folder
BATCH_SIZE = 32
EPOCHS = 64
VAL_SPLIT = 0.2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_NAMES = [
    # 'resnet50',
    # 'densenet121',
    'densenet169',
    'efficientnet_b0',
    'convnext_base',
    'vit_base_patch16_224',
    'swin_base_patch4_window7_224'
]
# Transforms (same for train and val to keep things simple)
common_transforms = transforms.Compose([
    transforms.Resize((224, 224)),

    # Randomly change brightness, contrast, saturation, and hue
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

    # Random horizontal and vertical flip
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),

    # Random rotation within ±30 degrees
    transforms.RandomRotation(degrees=30),

    # Random affine transformation (rotation, translation, scale, shear)
    transforms.RandomAffine(
        degrees=15,          # additional rotation control
        translate=(0.05, 0.05),  # 5% translation in both directions
        scale=(0.9, 1.1),    # zoom in/out
        shear=10             # shear angle
    ),

    # Random perspective transformation
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load full dataset and split
full_dataset = ImageFolder(DATA_DIR, transform=common_transforms)
NUM_CLASSES = len(full_dataset.classes)
val_size = int(len(full_dataset) * VAL_SPLIT)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Training and evaluation functions
def train_model(model, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return total_loss / total, 100. * correct / total

def evaluate_model(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# Freeze backbone parameters
def freeze_backbone(model):
    for param in model.parameters():
        param.requires_grad = False

# Build model with frozen backbone + classifier head
def build_feature_extractor(model_name):
    backbone = create_model(model_name, pretrained=True, num_classes=0, global_pool='avg')
    freeze_backbone(backbone)
    model = nn.Sequential(
        backbone,
        nn.Linear(backbone.num_features, NUM_CLASSES)
    )
    return model.to(DEVICE)


/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import mlflow
experiment_name = "backbone-features"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = None
if not experiment:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

: 

In [ ]:
# input_example = torch.rand(1, 3, 224, 224).numpy()
# Train loop for all models
for model_name in MODEL_NAMES:
    with mlflow.start_run(experiment_id=experiment_id, run_name=model_name):
        print(f"\n🔍 Training with frozen backbone: {model_name}")
        model = build_feature_extractor(model_name)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model[-1].parameters(), lr=1e-3)
        best_acc = 0

        for epoch in range(EPOCHS):
            print(f"Epoch {epoch+1}/{EPOCHS}")
            train_loss, train_acc = train_model(model, criterion, optimizer)
            val_acc = evaluate_model(model)
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
            mlflow.log_metrics({
                "train_accuracy": train_acc,
                "train_loss": train_loss,
                "val_accuracy": val_acc
            }, step=epoch)

            if val_acc > best_acc:
                best_acc = val_acc
                mlflow.pytorch.log_model(model, "model")


🔍 Training with frozen backbone: densenet169
Epoch 1/64


Train Loss: 4.9398, Train Acc: 6.72%, Val Acc: 26.16%


2025/06/08 22:02:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 2/64


Train Loss: 3.5456, Train Acc: 35.22%, Val Acc: 45.16%


...

Train Loss: 0.2081, Train Acc: 96.23%, Val Acc: 85.47%
Epoch 41/64


Train Loss: 0.1938, Train Acc: 97.15%, Val Acc: 85.66%
Epoch 42/64


Train Loss: 0.1990, Train Acc: 96.86%, Val Acc: 85.47%
Epoch 43/64


Train Loss: 0.1941, Train Acc: 96.76%, Val Acc: 82.95%
Epoch 44/64


Training:  25%|██▍       | 16/65 [00:29<01:19,  1.62s/it]